In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [2]:
credit_card_data = pd.read_csv('creditcard.csv')

# Preprocessing
scaler = StandardScaler()
credit_card_data['Normalized_Amount'] = scaler.fit_transform(credit_card_data['Amount'].values.reshape(-1, 1))
credit_card_data = credit_card_data.drop(['Time', 'Amount'], axis=1)

In [ ]:
credit_card_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15936 entries, 0 to 15935
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    15936 non-null  int64  
 1   V1      15936 non-null  float64
 2   V2      15936 non-null  float64
 3   V3      15936 non-null  float64
 4   V4      15936 non-null  float64
 5   V5      15936 non-null  float64
 6   V6      15936 non-null  float64
 7   V7      15936 non-null  float64
 8   V8      15936 non-null  float64
 9   V9      15936 non-null  float64
 10  V10     15936 non-null  float64
 11  V11     15936 non-null  float64
 12  V12     15936 non-null  float64
 13  V13     15936 non-null  float64
 14  V14     15936 non-null  float64
 15  V15     15936 non-null  float64
 16  V16     15936 non-null  float64
 17  V17     15936 non-null  float64
 18  V18     15936 non-null  float64
 19  V19     15936 non-null  float64
 20  V20     15936 non-null  float64
 21  V21     15936 non-null  float64
 22

In [17]:
credit_card_data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,Normalized_Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.0,0.335315
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.0,-0.272798
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0.0,1.283265
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.0,0.227210
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.0,0.005743


In [18]:
credit_card_data.tail()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,Normalized_Amount
1981,-0.340622,1.132232,1.291494,0.062313,0.016387,-0.977070,0.723755,-0.074630,-0.396655,-0.514041,...,-0.262948,-0.688785,-0.010937,0.334061,-0.160025,0.071779,0.245128,0.098336,0.0,-0.261789
1982,-1.842696,1.740641,0.861526,-0.856315,-0.655376,-0.842786,0.198563,0.602764,0.455595,0.541296,...,-0.213609,-0.400617,0.030013,0.512611,-0.077087,0.286218,0.586012,0.352610,0.0,-0.279793
1983,-0.480693,0.646091,1.577264,-0.084411,-0.305958,-0.534739,0.860346,-0.028569,-0.800705,-0.420592,...,0.121681,0.175190,0.035986,0.557665,-0.112301,0.337154,-0.015602,0.051504,0.0,0.050069
1984,-0.342132,1.091125,1.282729,0.068076,-0.022498,-0.996727,0.676304,-0.042250,-0.312036,-0.495759,...,-0.269850,-0.734148,-0.007354,0.319161,-0.179146,0.073683,0.241932,0.097139,0.0,-0.269073
1985,-0.854343,1.382948,1.278665,2.914727,-0.183139,-0.349329,0.274566,0.435277,-1.576521,0.645463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
legit = credit_card_data[credit_card_data['Class'] == 0]
fraud = credit_card_data[credit_card_data['Class'] == 1]
legit_sample = legit.sample(n=len(fraud))
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [4]:
X = new_dataset.drop('Class', axis=1)
Y = new_dataset['Class']

In [5]:
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]
print(legit.shape)

fraud = credit_card_data[credit_card_data.Class == 1]
print(legit.shape)
print(fraud.shape)

(1983, 30)
(1983, 30)
(2, 30)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [8]:
credit_card_data.groupby('Class').mean()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Normalized_Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,-0.281494,0.267200,0.848906,0.146804,-0.077904,0.051713,0.139533,-0.059771,0.014492,0.017172,...,0.056618,-0.012217,-0.144666,-0.043548,0.013865,0.108318,0.049441,0.027220,-0.001966,-0.000818
1.0,-2.677884,-0.602658,-0.260694,3.143275,0.418809,-1.245684,-1.105907,0.661932,-1.520521,-1.805429,...,1.114625,0.589464,0.200214,0.455377,0.013198,0.162159,0.016239,0.004186,-0.053756,0.810780


In [ ]:
legit_sample = legit.sample(n=492)

In [9]:
X_train_lstm = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)



In [10]:
model = Sequential([
    LSTM(64, input_shape=(X_train_lstm.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
model.fit(X_train_lstm, Y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.7022 - accuracy: 0.5000 - val_loss: 0.7192 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 72ms/step - loss: 0.6822 - accuracy: 0.5000 - val_loss: 0.7267 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 75ms/step - loss: 0.6658 - accuracy: 0.5000 - val_loss: 0.7342 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 74ms/step - loss: 0.6830 - accuracy: 0.5000 - val_loss: 0.7424 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 73ms/step - loss: 0.6914 - accuracy: 0.5000 - val_loss: 0.7506 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 58ms/step - loss: 0.6678 - accuracy: 0.5000 - val_loss: 0.7589 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 74ms/step - loss: 0.6913 - accuracy: 0.5000 - val_loss: 0.7673 - val_accuracy: 0.0000e+00
E

In [13]:
test_loss, test_acc = model.evaluate(X_test_lstm, Y_test)
print('Test Accuracy:', test_acc)

1/1 [==============================] - 0s 29ms/step - loss: 0.6476 - accuracy: 1.0000
Test Accuracy: 1.0


In [14]:
print("test loss : ",test_loss)

test loss :  0.6475695371627808
